# FACETpy Full Correction Workflow (Factory Syntax)

This tutorial shows a complete artifact-correction workflow using `create_standard_pipeline(...)`.

You will: configure the factory pipeline, inspect generated steps, run correction, review metrics, and verify outputs.

In [ ]:
from pathlib import Path

from facet import create_standard_pipeline

project_root = Path.cwd()
if not (project_root / "examples").exists() and (project_root.parent / "examples").exists():
    project_root = project_root.parent

input_file = project_root / "examples" / "datasets" / "NiazyFMRI.edf"
output_dir = project_root / "output" / "notebooks"
output_dir.mkdir(parents=True, exist_ok=True)

output_file = output_dir / "corrected_factory.edf"
plot_file = output_dir / "factory_before_after.png"

print(f"Input file: {input_file}")
assert input_file.exists(), f"Missing dataset: {input_file}"

## Step 1: Build the Standard Pipeline

The factory creates a full correction pipeline with recommended defaults: trigger detection, acquisition handling, filtering, AAS correction, optional PCA, evaluation, plotting, and export.

`use_anc=False` keeps this tutorial portable even when the ANC C extension is not available.

In [ ]:
pipeline = create_standard_pipeline(
    input_path=str(input_file),
    output_path=str(output_file),
    trigger_regex=r"\\b1\\b",
    artifact_to_trigger_offset=-0.005,
    upsample_factor=10,
    use_anc=False,
    use_pca=True,
    evaluate=True,
    plot=True,
    plot_kwargs={
        "mode": "matplotlib",
        "channel": "Fp1",
        "start": 25.0,
        "duration": 20.0,
        "overlay_original": True,
        "show": False,
        "auto_close": True,
        "save_path": str(plot_file),
        "title": "Factory Workflow: Before vs After",
    },
)

print(pipeline)

## Step 2: Inspect the Generated Processors

This makes the factory transparent: you can see exactly which processors will run and in what order.

In [ ]:
for idx, processor in enumerate(pipeline.processors, start=1):
    print(f"{idx:02d}. {processor.name}")

## Step 3: Run Full Correction

Running the pipeline executes the complete workflow end to end and returns a `PipelineResult` object with context, status, runtime, and metrics.

In [ ]:
result = pipeline.run()
result.print_summary()

## Step 4: Review Correction Quality Metrics

These metrics quantify how correction changed signal quality (for example SNR and RMS-based measures).

In [ ]:
result.print_metrics()

metrics = result.metrics
print("\nKey metrics:")
for key in ("snr", "legacy_snr", "rms_ratio", "rms_residual", "median_artifact"):
    if key in metrics:
        print(f"  {key}: {metrics[key]}")

## Step 5: Confirm Output Files

The workflow should produce both a corrected EDF file and a before/after comparison plot.

In [ ]:
print(f"Corrected file: {output_file}")
print(f"Exists: {output_file.exists()}")
print(f"Plot file: {plot_file}")
print(f"Exists: {plot_file.exists()}")